In [1]:
from src import *
from verde_source import regular, interp_at

from tqdm import tqdm
from shapely.ops import transform
from pylab import *
from shapely.geometry import Point, Polygon

import verde as vd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import geopandas as gpd
import pyproj
import os

/home/ggrl/.config/ambiente_geologico/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

%matplotlib widget

# Construindo Quadrícula

In [3]:
quadricula = Build_mc(escala='50k',ID=['SF23_YA','SF23_YB','SF23_VC','SF23_VD'],verbose=True)

100%|██████████| 4/4 [00:00<00:00, 210.62it/s]
97it [00:00, 12956.93it/s]

 - Folha "SF23_YA_I1" adicionada.
 - Folha "SF23_YA_I3" adicionada.
 - Folha "SF23_YA_IV1" adicionada.
 - Folha "SF23_YA_IV3" adicionada.
 - Folha "SF23_YA_I2" adicionada.
 - Folha "SF23_YA_I4" adicionada.
 - Folha "SF23_YA_IV2" adicionada.
 - Folha "SF23_YA_IV4" adicionada.
 - Folha "SF23_YA_II1" adicionada.
 - Folha "SF23_YA_II3" adicionada.
 - Folha "SF23_YA_V1" adicionada.
 - Folha "SF23_YA_V3" adicionada.
 - Folha "SF23_YA_II2" adicionada.
 - Folha "SF23_YA_II4" adicionada.
 - Folha "SF23_YA_V2" adicionada.
 - Folha "SF23_YA_V4" adicionada.
 - Folha "SF23_YA_III1" adicionada.
 - Folha "SF23_YA_III3" adicionada.
 - Folha "SF23_YA_VI1" adicionada.
 - Folha "SF23_YA_VI3" adicionada.
 - Folha "SF23_YA_III2" adicionada.
 - Folha "SF23_YA_III4" adicionada.
 - Folha "SF23_YA_VI2" adicionada.
 - Folha "SF23_YA_VI4" adicionada.
 - Folha "SF23_YA_VI2" adicionada.
 - Folha "SF23_YB_I1" adicionada.
 - Folha "SF23_YB_I3" adicionada.
 - Folha "SF23_YB_IV1" adicionada.
 - Folha "SF23_YB_IV3" adi

# Adicionando dados brutos à Quadrícula

In [ ]:
list_geof = os.listdir('/home/ggrl/database/geof/')
list_geof

In [ ]:
gama_3022,mag_3022=Upload_geof(quadricula,'gama_3022','mag_3022',1100)
gama_1105,mag_1105=Upload_geof(quadricula,'gama_line_1105','mag_line_1105',1100)
gama_1039,mag_1039=Upload_geof(quadricula,'gama_1039','mag_1039',1100)

In [ ]:
geof_list_ids = list(quadricula.keys())
print(len(geof_list_ids))
for id in geof_list_ids:
    print(f' - Folha: {id}')
    carta=quadricula[id]
    for data in list(carta.keys())[2:]:
        print(f'    - {data}')

In [ ]:
quadricula=pop_nodata(quadricula)
len(quadricula.keys())

In [ ]:
for id in list(quadricula.keys()):
    print(f' - Folha:  {id}')
    carta = quadricula[id]
    print(f'    - {list(carta.keys())}')

## Vizualisando Área de Estudo

In [ ]:
plt.figure(figsize=(16,9))
for id in list(quadricula.keys()):
    carta=quadricula[id]
    plt.plot(*transform_to_carta_utm(carta['folha']).exterior.xy,color='black')
    for data in list(carta.keys())[2:]:
        if 'mag' in data:
            pass
        else:
            plt.scatter(carta[data].X,carta[data].Y,c=carta[data].MDT,cmap='terrain',s=0.5,marker='H')
            plt.axis('scaled')
plt.suptitle('Área de cobertura dos levantamentos aerogeofísicos')
plt.tight_layout()

## Visualizando dados Radiométricos Brutos

In [ ]:
plot_histograms(gama_3022)

In [ ]:
plot_raw_gama_data(gama_3022,suptitle='Dados Radiométricos brutos (Gama_3022.XYZ)')

In [ ]:
plot_boxplots(gama_1105,gama_FEAT)

In [ ]:
plot_histograms(gama_1105)

In [ ]:
plot_raw_gama_data(gama_1105,suptitle='DADOS AEROGAMAESPECTROMÉTRICO')

In [ ]:
gama_1039.rename(columns={'CTC':'CTCOR','KC':'KPERC','UC':'eU','THC':'eTh'},inplace=True)
plot_histograms(gama_1039)
plot_raw_gama_data(gama_1039,suptitle='Dados Radiométricos brutos (XYZ)',orientation='vertical')

## Removendo valores negativos das contagens radiométricas

In [ ]:
gama_3022_positive = remove_negative_values(gama_3022)

In [ ]:
plot_boxplots(gama_3022_positive,gama_FEAT)

In [ ]:
plot_histograms(gama_3022_positive)

In [ ]:
plot_raw_gama_data(gama_3022_positive,'Dados radiométricos tratadas : value <= 0 == 0.001')

In [ ]:
gama_1105_positive = remove_negative_values(gama_1105)

In [ ]:
plot_histograms(gama_1105_positive)

In [ ]:
plot_boxplots(gama_1105_positive,gama_FEAT)

In [ ]:
plot_raw_gama_data(gama_1105_positive,'Dados radiométricos tratadas : value <= 0 == 0.001')

In [ ]:
gama_1039_positive=remove_negative_values(gama_1039,lista=['X','Y','LATITUDE','LONGITUDE','geometry'])
gama_1039_positive['UTHRAZAO']=gama_1039_positive['eU']/gama_1039_positive['eTh']
gama_1039_positive['UKRAZAO']=gama_1039_positive['eU']/gama_1039_positive['KPERC']
gama_1039_positive['THKRAZAO']=gama_1039_positive['eTh']/gama_1039_positive['KPERC']

plot_histograms(gama_1039_positive)
plot_raw_gama_data(gama_1039_positive,'Dados radiométricos tratadas : value <= 0 == 0.001')

## CONSTRUINDO UM GRID SINTÉTICO

In [ ]:
# REMOVING PART OF THE SINTETIC GRID
'''
df_xu_yu = pd.DataFrame(np.array([xu,yu]))
df_xu_yu=df_xu_yu.T
df_xu_yu.rename(columns={0:'xu',1:'yu'},inplace=True)

df_xu_yu[(df_xu_yu.xu < 540937) & (df_xu_yu.yu > 8866937)]

df_xu_yu.drop(df_xu_yu[(df_xu_yu.xu < 540937) & (df_xu_yu.yu > 8866937)].index,inplace=True)
plt.figure(figsize=(18,12))

plt.scatter(df_xu_yu.xu,df_xu_yu.yu,s=0.1,marker='.')
plt.axis('scaled')
'''



# Interpolação dos dados Brutos

## Método Cúbico

In [ ]:
# Test de área

# area=(344093.45426573796, 396417.36691108724, 7621768.799495494, 7677527.304557458)
# int((area[3]-area[2])/100),int((area[1]-area[0])/100)

In [ ]:
#traditional_interpolation(quadricula,'mag_3022','gama_3022','cubic','geof_3022')

In [ ]:
#list(quadricula['SF23_VC'].keys())

In [ ]:

#df = quadricula['SF23_VC']['geof_3022_cubic']
#plt.figure(figsize=(12,12))
#plt.scatter(x=df.X,y=df.Y,c=df.GMT,cmap='rainbow')
#plt.axis('scaled')

In [ ]:
# Print the output. a=

#descript_cubic = df.describe(percentiles)
#descript_cubic[['eU','eTh','KPERC','CTCOR','UTHRAZAO','THKRAZAO','UKRAZAO']].T

In [ ]:
#plot_histograms(geof_1089_cubic,suptitle='Distribuição dos dados radiométricos interpolados (cúbico, pixel 100m)')
#plot_raw_data(geof_1089_cubic,suptitle='Dados radiométricos interpolados (cúbico, pixel 100m)')

## Método Nearest

In [ ]:
#plot_histograms(geof_1089_nearest,suptitle='Distribuição dos dados radiométricos interpolados (nearest, pixel 100m)')

In [ ]:
#plot_raw_data(geof_1089_nearest,suptitle='Dados radiométricos interpolados (nearest, pixel 100m)')

## Método Linear

In [ ]:
traditional_interpolation(quadricula,'mag_3022','gama_3022','linear','geof_3022')

In [ ]:
traditional_interpolation(quadricula,'mag_line_1105','gama_line_1105','linear','geof_1105')

In [ ]:
plt.figure(figsize=(24,16))

# PLOTANDO A MALHA CARTOGRÁFICA
for id in list(quadricula.keys()):
    carta=quadricula[id]
    plt.plot(*transform_to_carta_utm(carta['folha']).exterior.xy,color='black')
    
    # PLOTANDO OS DADOS INTERPOLADOS
    for data in list(carta.keys())[2:]:        
        if 'geof' in data:
            plt.scatter(carta[data].X,carta[data].Y,c=carta[data].eU,cmap='rainbow',s=0.5,marker='H')
            plt.axis('scaled')
        # SE NÃO TIVER DADOS NÃO PLOTA NADA
        else:
            pass
        
plt.suptitle('Área de cobertura dos levantamentos aerogeofísicos')
plt.tight_layout()

In [ ]:
traditional_interpolation(quadricula,'mag_1039','gama_1039','linear','geof_1039')

In [ ]:
'''
plt.figure(figsize=(24,16))

for id in list(quadricula.keys()):
    carta=quadricula[id]
    plt.plot(*transform_to_carta_utm(carta['folha']).exterior.xy,color='black')
    
    for data in list(carta.keys())[2:]:
        if 'geof' in data:
            plt.scatter(carta[data].X,carta[data].Y,c=carta[data].MDT,cmap='terrain',s=0.5,marker='H')
            plt.axis('scaled')
        else:
            pass
        
plt.suptitle('Área de cobertura dos levantamentos aerogeofísicos')
plt.tight_layout()
'''

# Classificações Não-Supervisionadas

## Self-organizing maps (SOM)

In [ ]:
import matplotlib
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn_som.som import SOM

def plot_corr(dataframe, size=10):
    plt.figure(figsize = (size+size*0.2, size), facecolor='w')
    corrMatrix = dataframe.corr()
    sns.heatmap(np.round(corrMatrix,2), annot=True)
    plt.xticks(rotation=90, ha='right')
    plt.yticks(rotation=0, ha='right')


### teste

In [ ]:
df = quadricula['SF23_YB']['geof_1105_linear']

plot_histograms(df)

plot_boxplots(df,FEAT)

In [ ]:
df

In [ ]:
plot_raw_gama_data(df,suptitle='Dados Radiométricos interpolados (Algoritmo: Linear)',figsize=(27,16))
plot_raw_mag_data(df,suptitle='Dados Magnetométricos interpolados (Algoritmo: Linear)')

In [ ]:
df = quadricula['SF23_YB']['geof_1105_linear']
plot_histograms(df)

#### plot_raw_gama_data(df,suptitle='Dados Radiométricos interpolados (Algoritmo: Linear)',figsize=(27,16))

In [ ]:
df

In [ ]:
df.rename(columns={'GMT':'MAGIGRF'},inplace=True)
plot_raw_mag_data(df,suptitle='Dados Magnetométricos interpolados (Algoritmo: Linear)')

## Pixel size

In [ ]:
df_rs = df
df_rs.rename(columns={'X':'E_utm','Y':'N_utm'},inplace=True)
#df_rs.fillna(0,inplace=True)

xpixel_size = (df_rs.E_utm.max()-df_rs.E_utm.min())/df_rs.E_utm.unique().size
ypixel_size = (df_rs.N_utm.max()-df_rs.N_utm.min())/df_rs.N_utm.unique().size
print('x:', xpixel_size, 'y:', ypixel_size)

nx=df_rs.E_utm.unique().size
ny=df_rs.N_utm.unique().size
ratio=ny/nx
xs = df_rs.E_utm.values.reshape(ny, nx)
ys = df_rs.N_utm.values.reshape(ny, nx)

In [ ]:
features = list(df_rs.columns[2:])
plot_corr(df_rs[features], size=11)
#print(features)
#plt.savefig('figs/correlation_matrix.png', dpi=400, bbox_inches='tight')

In [ ]:
data = StandardScaler().fit_transform(df[features].values)

# data = df_rs[features].values

# NÚMERO DE CLASSES
n_clusters=9
lito_SOM = SOM(m=n_clusters,
               n=1,
           sigma=1.5,
             dim=len(features),
        max_iter=10000)

lito_SOM.fit(data)

# predição de classes
predictions = lito_SOM.predict(data)

# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']
    
# classes weights
fig, ax = plt.subplots(figsize=(19,19))
im=ax.matshow(lito_SOM.weights)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(10, 10)
plt.show()

In [ ]:
id_ = [1,2,3,4,5,6,7,8,9]

relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)

idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)

datafig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',cmap='viridis',ticks=ticks)

cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.suptitle('SOM - Aerogeophysical Data (SF23_YA_III4)')
plt.axis('scaled')
plt.show()

In [ ]:
n_clusters=18
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.5, dim=len(features), max_iter=10000)
lito_SOM.fit(data)

# predição de classes
predictions = lito_SOM.predict(data)

# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']
    
# classes weights
fig, ax = plt.subplots(figsize=(27,27))
im=ax.matshow(lito_SOM.weights)

for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=8)
plt.xticks(range(len(features)), features, rotation=55, fontsize=9, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal',cmap='Reds')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(15, 10)
plt.show()

In [ ]:
df

In [ ]:
id_ = [1,2,3,4,5,6,7,8,9,10,11]

relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)

idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)

datafig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.suptitle('SOM - Aerogeophysical Data ()')
plt.axis('scaled')
plt.show()

## Testes

In [ ]:
df = quadricula['SF23_YA']['geof_1105_linear']

plot_histograms(df)
plot_raw_gama_data(df,suptitle='Dados Radiométricos interpolados (Algoritmo: Linear)',figsize=(27,16))
plot_raw_mag_data(df,suptitle='Dados Magnetométricos i4nterpolados (Algoritmo: Linear)')


df_rs = df
df_rs.rename(columns={'X':'E_utm','Y':'N_utm'},inplace=True)
#df_rs.fillna(0,inplace=True)


xpixel_size = (df_rs.E_utm.max()-df_rs.E_utm.min())/df_rs.E_utm.unique().size
ypixel_size = (df_rs.N_utm.max()-df_rs.N_utm.min())/df_rs.N_utm.unique().size
print('x:', xpixel_size, 'y:', ypixel_size)


nx=df_rs.E_utm.unique().size
ny=df_rs.N_utm.unique().size
ratio=ny/nx
xs = df_rs.E_utm.values.reshape(ny, nx)
ys = df_rs.N_utm.values.reshape(ny, nx)

features = list(df_rs.columns[2:])
print(features)


plot_corr(df_rs[features], size=12)
#plt.savefig('figs/correlation_matrix.png', dpi=400, bbox_inches='tight')



scaler = StandardScaler()
data = scaler.fit_transform(df[features].values)
# data = df_rs[features].values


n_clusters=9
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.5, dim=len(features), max_iter=10000)
lito_SOM.fit(data)

# predição de classes
predictions = lito_SOM.predict(data)

In [ ]:
df

In [ ]:
# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']
    
# classes weights
fig, ax = plt.subplots(figsize=(19,26))
im=ax.matshow(lito_SOM.weights)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(19, 26)
plt.show()

In [ ]:
id_ = [1,2,3,4,5,6,7,8,9]
relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)



idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)

datafig, ax=plt.subplots(figsize=(16, 16), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.suptitle('SOM - Aerogeophysical Data (SF23_YA_III4)')
plt.axis('scaled')
plt.show()

# Classificações Supervisionadas

## Rotulando amostras com classes litológicas

In [ ]:
import shapely.speedups
from shapely import geometry
shapely.speedups.enable()

geof_1089_linear['geometry'] = [geometry.Point(x,y) for x, y in zip(geof_1089_linear['X'], geof_1089_linear['Y'])]
gdf_1089_linear = geof_1089_linear.set_geometry('geometry')

gdf_1089_linear.set_crs('EPSG:32723',inplace=True)
gdf_1089_linear.geometry

In [ ]:
Upload_litologia(quadricula,'litologia_100k')

In [ ]:
litologia=quadricula['SB24_ZB_II']['litologia_100k']
litologia.to_crs('EPSG:32724',inplace=True)
print(litologia.crs)
litologia.reset_index(drop=True,inplace=True)

dic_litologico = describe_geologico(litologia)
print(litologia.columns)

In [ ]:
print(dic_litologico['SIGLA']['len'])
print(dic_litologico['SIGLA']['lista'])
gdf_1089_linear

In [ ]:
litologia.plot('SIGLA',figsize=(16,16),legend=True)

In [ ]:
geof_1089_linear['closest_unit'] = geof_1089_linear['geometry'].apply(lambda x: litologia['SIGLA'].iloc[litologia.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

In [ ]:
    geof_1089_linear.to_csv('/home/ggrl/database/csv/SB24_ZB_II_gama_linear_100m.csv',index=False)